# Correlation Coefficients

In [ ]:
from datascience import *
from cs104 import *
import numpy as np
%matplotlib inline

## 1. Finch data

In [ ]:
# Load finch data 
finch_1975 = Table().read_table("data/finch_beaks_1975.csv")
finch_1975.show(6)

In [ ]:
fortis = finch_1975.where('species', 'fortis')
fortis.num_rows

In [ ]:
scandens = finch_1975.where('species', 'scandens')
scandens.num_rows

In [ ]:
plot = fortis.scatter('Beak length, mm', 'Beak depth, mm')
plot.set_title('Fortis Finches, 1975')

In [ ]:
plot = fortis.scatter('Beak length, mm', 'Beak depth, mm', fit_line=True)
plot.set_title('Fortis Finches, 1975') 

In [ ]:
finch_1975.scatter('Beak length, mm', 'Beak depth, mm', group='species')
plot.set_title('Fortis Finches, 1975') 

## 2. Correlation Intuition

Visualize different values of r:

In [ ]:

def make_table(r, x_mean, y_mean, size, seed=10):
    """
    Make a table of size random (x,y) points with a 
    correlation coefficient of approximately r.
    The points will be centered at (x_mean,y_mean).
    """
    rng = np.random.default_rng(seed)
    x = rng.normal(x_mean, 1, size)
    z = rng.normal(y_mean, 1, size)
    y = r*x + (np.sqrt(1-r**2))*z
    
    # Make sure the mean is *exactly* what we want :).
    table = Table().with_columns("x", x - np.mean(x) + x_mean, "y", y - np.mean(y) + y_mean)
    
    return table

def plot_table(table, color='C4', fit_line=True, **kwargs):
    """
    Plot a table of (x,y).
    """
    plot = table.scatter("x", "y",alpha=0.5, fit_line=fit_line, color=color, **kwargs)
    plot.line(x = 0, color='white', width=4, zorder=0.9)
    plot.line(y = 0, color='white', width=4, zorder=0.9)    
    plot.set_xlim(-4, 4)
    plot.set_ylim(-4, 4)
    return plot

def r_scatter(r):
    "Generate a scatter plot with a correlation approximately r"
    table = make_table(r, 0, 0, 500)
    plot = plot_table(table)
    plot.set_title('r = '+str(r))

In [ ]:
interact(r_scatter, r = Slider(-1,1,0.01))

Here are several scatter plots with representative values of r.

## Computing r

Here is a small data set we'll use to develop a formula for computing Pearson's correlation coefficient $r$.

In [ ]:
tiny_data = make_table(0.4, 0, 0, 200)
plot_table(tiny_data, s=50, fit_line=False)

In [ ]:

def plot_table_quadarants(table):
    x = table.column("x")
    y = table.column("y")    
    cm = plots.cm.get_cmap('RdBu')
    plot = table.scatter("x", "y",alpha=0.85, color=None, c=x*y/abs(x*y), vmin=-1.3, vmax=1.3, s=50, cmap=cm)
    plot.line(x = 0, color='white', width=4, zorder=0.9)
    plot.line(y = 0, color='white', width=4, zorder=0.9)    
    plot.set_xlim(-4, 4)
    plot.set_ylim(-4, 4)
    return plot

def plot_table_magnitudes_broken(table, diagonals = True, fit_line=False):
    x = table.column("x")
    y = table.column("y")
    x_mean = 0
    y_mean = 0
    numerator = (x - x_mean) * (y - y_mean)
    denominator = np.sqrt(sum((x - x_mean)**2)) * np.sqrt(sum((y - y_mean)**2))
    
    limit = np.max(np.abs(numerator/denominator))
    
    cm = plots.cm.get_cmap('RdBu')
    plot = table.scatter("x", "y",fit_line=fit_line, alpha=0.75, color=None, c=(numerator/denominator), s=50, cmap=cm,  
                           norm=colors.SymLogNorm(linthresh=0.001,
                                              vmin=-limit, vmax=limit, base=10))
    plot.set_xlim(-4, 4)
    plot.set_ylim(-4, 4)
    plot.line(x = 0, color='white', width=4, zorder=0.9)
    plot.line(y = 0, color='white', width=4, zorder=0.9)
    if diagonals:
        plot.line(slope=1,intercept=0, color='gray', linestyle='--', width=2, zorder=0.92)    
        plot.line(slope=-1,intercept=0, color='gray', linestyle='--', width=2, zorder=0.92)    
    return plot

def plot_table_magnitudes(table, diagonals = True):
    x = table.column("x")
    y = table.column("y")
    x_mean = np.mean(x)
    y_mean = np.mean(y)    
    numerator = (x - x_mean) * (y - y_mean)
    denominator = np.sqrt(sum((x - x_mean)**2)) * np.sqrt(sum((y - y_mean)**2))
    
    limit = np.max(np.abs(numerator/denominator))
    
    cm = plots.cm.get_cmap('RdBu')
    plot = table.scatter("x", "y",alpha=0.75, color=None, c=(numerator/denominator), s=50, cmap=cm,  
                           norm=colors.SymLogNorm(linthresh=0.001,
                                              vmin=-limit, vmax=limit, base=10))
    plot.set_xlim(-4, 4)
    plot.set_ylim(-4, 4)
    plot.line(x = 0, color='white', width=4, zorder=0.9)
    plot.line(y = 0, color='white', width=4, zorder=0.9)
    if diagonals:
        plot.line(slope=1,intercept=0, color='gray', linestyle='--', width=2, zorder=0.92)    
        plot.line(slope=-1,intercept=0, color='gray', linestyle='--', width=2, zorder=0.92)    
    return plot
       

def pearson_correlation1(table, x_label, y_label):
    """
    Return the correlation coefficient capturing the sign
    and strength of the association between the given columns in the
    table.
    """
    x = table.column(x_label)
    y = table.column(y_label)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    r = sum(x * y) 
    return r

def pearson_correlation2(table, x_label, y_label):
    """
    Return the correlation coefficient capturing the sign
    and strength of the association between the given columns in the
    table.
    """
    x = table.column(x_label)
    y = table.column(y_label)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = sum(x * y) 
    denominator = np.sqrt(sum(x**2)) * np.sqrt(sum(y**2))
    return numerator / denominator 

def pearson_correlation2_denom(table, x_label, y_label):
    """
    Return the correlation coefficient capturing the sign
    and strength of the association between the given columns in the
    table.
    """
    x = table.column(x_label)
    y = table.column(y_label)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = sum(x * y) 
    denominator = np.sqrt(sum(x**2)) * np.sqrt(sum(y**2))
    return denominator 


def pearson_correlation(table, x_label, y_label):
    """
    Return the correlation coefficient capturing the sign
    and strength of the association between the given columns in the
    table.
    """
    x = table.column(x_label)
    y = table.column(y_label)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = sum((x - x_mean) * (y - y_mean)) 
    denominator = np.sqrt(sum((x - x_mean)**2)) * np.sqrt(sum((y - y_mean)**2))
    return numerator / denominator 

### Step 1

We first note that the blue points in the top-right and bottom-left quadrants reflect positive correlation between x and y.  The red points in the other two quadrents reflect negative correlation.

In [ ]:
plot_table_quadarants(tiny_data)

We can determined whether a point $(x,y)$ contributes to a positive/negative association by inspecting the sign if $x \cdot y$.  Also points close to the diagonals and further away from the origin reflect much stronger correlation than those near the origin.

In [ ]:
plot_table_magnitudes_broken(tiny_data)

This leads us to our first way to quantify correlation:
$
\sum_i x_i y_i = x_0 y_0 + x_1 y_1 + \ldots + x_n y_n
$.

However, this formula can produce arbitrarily large or small numbers.  For our sample data, that formula produces 93.1.  

We must normalize it to be in the range -1 to 1 to ensure our coefficient is interpretable:

$$
\frac{\sum_i x_i y_i}{\sqrt{\sum_i x_i^2}\sqrt{\sum_i y_i^2}} 
$$ 

With that adjustment, our data set yields 0.48.

In [ ]:

x_bar = 1
y_bar = 1.5

uncentered = make_table(-0.65, x_bar, y_bar, 200, 13)
centered = make_table(-0.65, 0, 0, 200, 13)

Now consider the data set on the left below.  Unlike those above, this data is not "centered" at (0,0).  That is the mean $x$ and mean $y$ values are not both 0.  Our formula produces the correlation coefficient 0.24, but $x$ and $y$ are not positively correlated, but rather negatively!  The issue can be seen with the plot on the right: we've mis-classified many points as showing positive correlation instead of negative because where they line on the plane.

In [ ]:
with Figure(1,2):
    plot = plot_table(uncentered, s=50, color='C2', fit_line=False)
    plot = plot_table_magnitudes_broken(uncentered, diagonals=False)

To handle this, we must recenter that data at (0,0) by subtract the mean x and mean y values $(\bar{x},\bar{y})$ from each point with our final formula for $r$:

$$
r = \frac{
  \sum_i (x_i - \bar{x})(y_i - \bar{y})
}{
  \sqrt{\sum_i(x_i - \bar{x})^2}\sqrt{\sum_i(y_i - \bar{y})^2}
}
$$

We see that adjustment visually below, and the formula above shows $r$ to be -0.70.

In [ ]:
with Figure(1,2):
    plot = plot_table(uncentered, s=50, color='C2', fit_line=False)
    plot.dot(x_bar, y_bar, color='C2')
    
    plot = plot_table(centered, s=50, color='C2', fit_line=True)
    plot.dot(0, 0, color='C2')    

## 3. Implementing Pearson's Correlation Coefficient

In [ ]:
def pearson_correlation(table, x_label, y_label):
    """
    Return the correlation coefficient capturing the sign
    and strength of the association between the given columns in the
    table.
    """
    x = table.column(x_label)
    y = table.column(y_label)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = sum((x - x_mean) * (y - y_mean)) 
    denominator = np.sqrt(sum((x - x_mean)**2)) * np.sqrt(sum((y - y_mean)**2))
    return numerator / denominator 

In [ ]:
fortis_r = pearson_correlation(fortis, 'Beak length, mm', 'Beak depth, mm')
fortis_r

In [ ]:
scandens_r = pearson_correlation(scandens, 'Beak length, mm', 'Beak depth, mm')
scandens_r

In [ ]:
finch_1975.scatter('Beak length, mm', 'Beak depth, mm', fit_line=True, group="species")

In [ ]:
fortis_r = pearson_correlation(fortis, 'Beak length, mm', 'Beak depth, mm')
fortis_r

In [ ]:
scandens_r = pearson_correlation(scandens, 'Beak length, mm', 'Beak depth, mm')
scandens_r

In [ ]:
finch_1975.scatter('Beak length, mm', 'Beak depth, mm', fit_line=True, group="species")

Switching axes shouldn't change anything. 

In [ ]:
scandens_r = pearson_correlation(scandens, 'Beak depth, mm', 'Beak length, mm')
scandens_r

## 4. Watch out for... 

### Nonlinearity

In [ ]:
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=50, color='red')

In [ ]:
pearson_correlation(nonlinear, 'x', 'y')

### Outliers

What can cause outliers?  What to do when you encounter them?

In [ ]:
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4,5),
        'y', make_array(1, 2, 3, 4,5)
    )
line.scatter('x', 'y', s=50, color='red')

In [ ]:
pearson_correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=50, color='red')

In [ ]:
pearson_correlation(outlier, 'x', 'y')

### False Correlations due to Data Aggregation

In [ ]:
sat2014 = Table.read_table('data/sat2014.csv').sort('State')
sat2014

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
pearson_correlation(sat2014, 'Critical Reading', 'Math')

In [ ]:
sat_fake = Table.read_table('data/sat-bogus.csv').sort('State')  

Here is some fake data to illustrate this:

In [ ]:
sat_fake_g = sat_fake.group('State', np.mean)

with Figure(1,2, figsize=(6,4)):
    sat_fake_g.scatter('Critical Reading mean', 'Math mean', fit_line=True)
    sat_fake.scatter('Critical Reading', 'Math', group='State')

# print(pearson_correlation(sat_fake_g, 'Critical Reading mean', 'Math mean'))
# print(pearson_correlation(sat_fake, 'Critical Reading', 'Math'))

r is 0.98 on the left, but -0.09 for the un-aggregated data on the right!